In [7]:
import pandas as pd
import numpy as np
temp = pd.read_excel('/home/wonseok/projects/2022_DATA_SYNTHESIS/data/raw/CLRC_TRTM_CASB.xlsx')

In [26]:
temp[['PT_SBST_NO', 'CSTR_CYCL_VL']].sort_values(by = ['PT_SBST_NO', 'CSTR_CYCL_VL']).head()

,PT_SBST_NO,CSTR_CYCL_VL
0,RN00000009,1
12,RN00000009,1
1,RN00000009,2
13,RN00000009,2
2,RN00000009,3


In [8]:
pt_num = list(pd.read_excel('/home/wonseok/projects/2022_DATA_SYNTHESIS/young_age/data/raw/D0_Handmade_ver1.1.xlsx')['PT_SBST_NO']) # under 50 patients number

In [9]:
len(pt_num)

1501

In [10]:
regn = temp[temp['PT_SBST_NO'].isin(pt_num)].drop(['CENTER_CD','IRB_APRV_NO'],axis =1)

In [11]:
regn['CSTR_END_YMD'] = [np.NaN]*len(regn)

In [12]:
def transform_format(data, syn = False, droped = False):
    
    counts = []  
    counts = list(data.groupby(by ='PT_SBST_NO').count()['CSTR_STRT_YMD'])
    #data =get_days(data)
    if droped == True:
        data.drop('PT_SBST_NO', axis = 1 ,inplace = True)
    np_input = data.to_numpy()
    
    input_form= []

    cur = 0
    for count in list(counts):
        input_form.append(np_input[cur:cur+count])
        cur += count
    input_form = np.array(input_form)
    
    return input_form

In [13]:
np_regn = transform_format(regn)

/tmp/ipykernel_7329/1802675613.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  input_form = np.array(input_form)


In [14]:
from datetime import datetime, timedelta


for i in range(len(np_regn)):
    
    new_end_days = np_regn[i].transpose()[1][1:]
    last = (pd.to_datetime(np_regn[i].transpose()[1][-1], format = '%Y%m%d')+ timedelta(weeks=2)).strftime("%Y%m%d")

    new_end_days = np.append(new_end_days,int(last))
    np_regn[i].transpose()[6] = new_end_days

In [15]:
len_arr = []
for i in range(len(np_regn)):
    len_arr.append(len(np.unique(np_regn[i].transpose()[4])))

In [16]:
snapshot = []
for num in pt_num:
    pt0 = []
    pt0.append(num)
    
    if num in list(temp['PT_SBST_NO'].unique()):
        for i in range(len(np_regn)):
            if(num == np_regn[i].transpose()[0][0]):
                cycle_col = []
                cycle = np_regn[i].transpose()[9]

                cycle_col = []
                cycle = np_regn[i].transpose()[9]

                cycle_info = []
                for j in range(len(cycle)):
                    try:
                        if(cycle[j]-cycle[j+1]) < 0:
                            cycle_info.append(np_regn[i][j])
                            
                        else:
                            cycle_info.append(np_regn[i][j])
                            cycle_col.append(cycle_info)
                            
                            cycle_info =[]
                    except:
                        cycle_info.append(np_regn[i][j])
                        cycle_col.append(cycle_info)
                    
                for k in range(len(cycle_col)):
                    try:
                        cycle_col = np.array(cycle_col)
                        cycle_col[k] = np.array(cycle_col[k])
                        pt0.append(np.unique(cycle_col[k].transpose()[4]))
                        if len(np.unique(cycle_col[k].transpose()[5])) == 1:
                            pt0.append(np.unique(cycle_col[k].transpose()[5])[0])
                        else:
                            print("ERROR :: Number of Therphy is not unique value where in 1 cylce")
                            break
                        pt0.append(np.unique(cycle_col[k].transpose()[8]))
                        pt0.append(cycle_col[k].transpose()[1][0])
                        pt0.append(cycle_col[k].transpose()[6][-1])
                        pt0.append(cycle_col[k].transpose()[9][-1])
                    except:
                        for _ in range(5):
                            pt0.append(np.NaN)
    else:
        for _ in range(40):
            pt0.append("Not Data")
    
    snapshot.append(pt0)
    

/tmp/ipykernel_7329/3614725511.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cycle_col = np.array(cycle_col)


In [17]:
new_col = []
new_col.append('PT_SBST_NO')
#new_col = []
for i in range(1,9):
    new_col.append('CSTR_REGN_NM_'+str(i))
    new_col.append('CSTR_PRPS_NT_'+str(i))
    new_col.append('CASB_CSTR_PRPS_NM_'+str(i))
    new_col.append('TRTM_CASB_STRT_YMD'+str(i))
    new_col.append('TRTM_CASB_CSTR_YMD2_'+str(i))
    new_col.append('CSTR_CYCL_VL_END_'+str(i))

new_d0 = pd.DataFrame(snapshot,columns=new_col)

In [18]:
new_d0

,PT_SBST_NO,CSTR_REGN_NM_1,CSTR_PRPS_NT_1,CASB_CSTR_PRPS_NM_1,TRTM_CASB_STRT_YMD1,TRTM_CASB_CSTR_YMD2_1,CSTR_CYCL_VL_END_1,CSTR_REGN_NM_2,CSTR_PRPS_NT_2,CASB_CSTR_PRPS_NM_2,...,CASB_CSTR_PRPS_NM_7,TRTM_CASB_STRT_YMD7,TRTM_CASB_CSTR_YMD2_7,CSTR_CYCL_VL_END_7,CSTR_REGN_NM_8,CSTR_PRPS_NT_8,CASB_CSTR_PRPS_NM_8,TRTM_CASB_STRT_YMD8,TRTM_CASB_CSTR_YMD2_8,CSTR_CYCL_VL_END_8
0,RN00001835,[FOLFOX],1,[Adjuvant],20080307,20080913,12,None,None,None,...,None,None,NaN,NaN,None,NaN,None,NaN,NaN,NaN
1,RN00001096,[FOLFOX],1,[Adjuvant],20080217,20080804,12,None,None,None,...,None,None,NaN,NaN,None,NaN,None,NaN,NaN,NaN
2,RN00005348,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
3,RN00003729,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
4,RN00015427,[FOLFOX],1,[Adjuvant],20080910,20090304,12,None,None,None,...,None,None,NaN,NaN,None,NaN,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496,RN00010979,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
1497,RN00015283,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
1498,RN00017333,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
1499,RN00009944,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN


[['RN00001835',
  array(['FOLFOX'], dtype=object),
  1,
  array(['Adjuvant'], dtype=object),
  20080307,
  20080913,
  12],
 ['RN00001096',
  array(['FOLFOX'], dtype=object),
  1,
  array(['Adjuvant'], dtype=object),
  20080217,
  20080804,
  12],
 ['RN00005348',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data'],
 ['RN00003729',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'Not Data',
  'No

In [67]:
set(map(lambda x : len(x), snapshot))

{7, 13, 19, 25, 31, 37, 41, 43, 49}

In [69]:
pd.DataFrame(snapshot, columns=new_col)

,PT_SBST_NO,CSTR_REGN_NM_1,CSTR_PRPS_NT_1,CASB_CSTR_PRPS_NM_1,TRTM_CASB_STRT_YMD1,TRTM_CASB_CSTR_YMD2_1,CSTR_CYCL_VL_END_1,CSTR_REGN_NM_2,CSTR_PRPS_NT_2,CASB_CSTR_PRPS_NM_2,...,CASB_CSTR_PRPS_NM_7,TRTM_CASB_STRT_YMD7,TRTM_CASB_CSTR_YMD2_7,CSTR_CYCL_VL_END_7,CSTR_REGN_NM_8,CSTR_PRPS_NT_8,CASB_CSTR_PRPS_NM_8,TRTM_CASB_STRT_YMD8,TRTM_CASB_CSTR_YMD2_8,CSTR_CYCL_VL_END_8
0,RN00001835,[FOLFOX],1,[Adjuvant],20080307,20080913,12,None,None,None,...,None,None,NaN,NaN,None,NaN,None,NaN,NaN,NaN
1,RN00001096,[FOLFOX],1,[Adjuvant],20080217,20080804,12,None,None,None,...,None,None,NaN,NaN,None,NaN,None,NaN,NaN,NaN
2,RN00005348,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
3,RN00003729,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
4,RN00015427,[FOLFOX],1,[Adjuvant],20080910,20090304,12,None,None,None,...,None,None,NaN,NaN,None,NaN,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496,RN00010979,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
1497,RN00015283,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
1498,RN00017333,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN
1499,RN00009944,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,Not Data,...,Not Data,Not Data,NaN,NaN,None,NaN,None,NaN,NaN,NaN


In [87]:
set(map(lambda x : len(x), snapshot))

{7, 13, 19, 25, 31, 37, 41, 43, 49}

In [90]:
len(new_col)

49